In [1]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install sentencepiece
! pip install transformers
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.5 MB/s eta 0:00:00


In [4]:
import os
from typing import Dict, Tuple

import datasets
import evaluate
import torch.cuda
from datasets import DatasetDict, load_dataset, Dataset
from evaluate import EvaluationModule
from torch import Tensor
from transformers import AutoConfig,AutoModelForSequenceClassification, TrainingArguments, Trainer,EvalPrediction
from transformers import AutoTokenizer, CamembertModel
import numpy as np
import datasets
from datasets import load_metric

# dataset preprocess

In [5]:
from typing import Any, Dict, Callable

# Just a place-holder type hint to label a data point in a dataset, e.g. train_dataset[0]
from datasets import DatasetDict, load_dataset

DataPoint = Dict[str, Any]


def lowercase_text(item: DataPoint) -> DataPoint:
    """
    Convert the text of the data point to lower case
    :param item: a datapoint (needs a text attribute)
    :return:
    """
    # notice! you can just operate on the attribute you care about, and not worry about deleting/preserving the other
    # attributes. This is due to the update semantics of datasets.map
    return {"text": item['premise'].lower()}

    # also acceptable: update AND return, e.g:
    # item['text'] = item['text'].lower()
    # return item


def pre_process_dataset(dataset: DatasetDict) -> DatasetDict:
    """
    Our complete pre-processing pipeline, returning the result as a new DatasetDict
    
    1. lower-case all input texts and replace new-lines with spaces
    2. remove training points labelled as part of the `misc` group in the training data (e.g. for zero-shot transfer)
    
    :param dataset: un-processed dataset
    :return: processed dataset
    """

    # Calling map on a DatasetDict applies to all splits
    # datasets do not update in place, allowing you to have several versions. Means you must assign result
    dataset = dataset.map(lowercase_text)
    assert 'label' in dataset['train'][0]  # note: non-destructive operation

    # you can also just pass a lambda. We'll do this for the new-lines replacement. 'desc' is a parameter passed for
    # tqdm, which updates labels on the progress bar.
    dataset = dataset.map(
        lambda item: {'text': ' '.join(item['premise'].split())},
        desc="normalizing all white space to a single space"
    )

    return dataset



# loading the dataset will download if not present in your cache, or simply load from there
dataset: DatasetDict = load_dataset("kseth919/snli-french")

# see function above for details
#dataset = pre_process_dataset(dataset)

# notice the reduced size due to filter
print(dataset)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/9842 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/549367 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9824 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/kseth919___parquet/kseth919--snli-french-7035369d5da6b1b0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 9842
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 549367
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 9824
    })
})


#evalution metric

In [6]:
from pprint import pprint
import evaluate
from evaluate import TextClassificationEvaluator, Metric, EvaluationModuleInfo
from sklearn.metrics import f1_score

class MyMacroF1Metric(Metric):
    """
    You can define custom metrics! In this case I do this to compute Macro-F1, which averages per-class F1 scores
    """
    f1_metric_info: EvaluationModuleInfo = evaluate.load("f1")._info()

    def _info(self) -> EvaluationModuleInfo:
        # we'll just say the info is the same in this case
        return MyMacroF1Metric.f1_metric_info

    def _compute(self, predictions=None, references=None, labels=None, pos_label=1, sample_weight=None) -> Dict[str, Any]:
        # we can just call the sklearn implementation! Metrics in huggingface generally correspond with sklearn metrics
        # when applicable
        score = f1_score(
            references, predictions, labels=labels, pos_label=pos_label, average="macro", sample_weight=sample_weight
        )
        return {"f1": float(score) if score.size == 1 else score}

#model


In [7]:
def split_to_train_and_valid(full_set: Dataset, proportion_train: float) -> Tuple[Dataset, Dataset]:
    train_and_dev: DatasetDict = full_set.train_test_split(train_size=proportion_train)
    return train_and_dev['train'], train_and_dev['test']

# load and pre-process our dataset
dataset: DatasetDict = load_dataset("kseth919/snli-french")
dataset = dataset.filter(lambda example: example['label'] != -1)
dataset = pre_process_dataset(dataset)

# instantiate tokenizer, model, and config
config: AutoConfig = AutoConfig.from_pretrained("cmarkea/distilcamembert-base")

# happens to use the BERT tokenizer! don't guess!
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained("cmarkea/distilcamembert-base")
model: AutoModelForSequenceClassification = AutoModelForSequenceClassification.from_pretrained(
            "cmarkea/distilcamembert-base",
            # we get a new classification head! Need to provide number of labels
            num_labels=3,
            # if we don't give a mapping of some kind, it will make up a strange one: {n: f"LABEL_{n}", ...}
            # We just consider labels to be the integers themselves and don't bother mapping back to meaningful classes
            id2label={i: i for i in range(3)}
        )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

  0%|          | 0/3 [00:00<?, ?it/s]

Filter:   0%|          | 0/9842 [00:00<?, ? examples/s]

Filter:   0%|          | 0/549367 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9824 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

normalizing all white space to a single space:   0%|          | 0/9842 [00:00<?, ? examples/s]

normalizing all white space to a single space:   0%|          | 0/549367 [00:00<?, ? examples/s]

normalizing all white space to a single space:   0%|          | 0/9824 [00:00<?, ? examples/s]

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['classifier.out_proj.bias', 'clas

In [ ]:
dataset_cache_path: str = "/content/drive/My Drive/nlp244/assignments/quest-4/tokenized_data"
if not os.path.exists(dataset_cache_path):
        # Let's first split training into train (80% of train) and dev (20% of train)
        test_dataset: Dataset = dataset['test']
        train_dataset, dev_dataset = split_to_train_and_valid(dataset['train'], proportion_train=0.8)

        # now let's tokenize and truncate. Keeping separate since we format slightly differently
        train_dataset = train_dataset.map(lambda batch: tokenizer(batch['premise'],batch['hypothesis'], truncation=True), batched=True, batch_size=256)
        dev_dataset = dev_dataset.map(lambda batch: tokenizer(batch['premise'],batch['hypothesis'],truncation=True), batched=True, batch_size=256)
        test_dataset = test_dataset.map(lambda batch: tokenizer(batch['premise'],batch['hypothesis'], truncation=True), batched=True, batch_size=256)
        train_dataset.save_to_disk(os.path.join(dataset_cache_path, "train_dataset"))
        dev_dataset.save_to_disk(os.path.join(dataset_cache_path, "dev_dataset"))
        test_dataset.save_to_disk(os.path.join(dataset_cache_path, "test_dataset"))
else:
        train_dataset = datasets.load_from_disk("/content/drive/My Drive/nlp244/assignments/quest-4/tokenized_data/train_dataset")
        dev_dataset = datasets.load_from_disk("/content/drive/My Drive/nlp244/assignments/quest-4/tokenized_data/dev_dataset")
        test_dataset = datasets.load_from_disk("/content/drive/My Drive/nlp244/assignments/quest-4/tokenized_data/test_dataset")


# convert train set to tensors with only model inputs
#train_dataset.set_format(type="pt", columns=['label', 'fr_premise', 'fr_hypothesis', 'input_ids', 'attention_mask'])

my_evaluation: MyMacroF1Metric = MyMacroF1Metric()

def my_compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
        logits, labels = eval_pred.predictions, eval_pred.label_ids
        predictions: Tensor = logits.argmax(axis=-1)
        return my_evaluation.compute(predictions=predictions, references=labels)

    # Let's fine-tune with the Trainer API!
training_args: TrainingArguments = TrainingArguments(
        output_dir="/content/drive/My Drive/nlp244/assignments/quest-4/checkpoints",
        do_train=True,
        do_eval=True,
        do_predict=True,
        evaluation_strategy="steps",
        eval_steps=128,
        per_device_train_batch_size=80,
        per_device_eval_batch_size=1280,
        save_steps=128,
        save_strategy="steps",
        save_total_limit=5,
        #report_to=["wandb"],
        logging_steps=50,
        num_train_epochs=20,
        learning_rate=0.001,
        metric_for_best_model="f1",
        load_best_model_at_end=True,
        max_grad_norm = 0.5,
        dataloader_num_workers=0,  # set to 0 when debugging and >1 when running!
    )


#wandb.init(entity="kingb12", project="nlp244-hf-libraries-demo", group="finetune_w_trainer")

trainer: Trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=None,  # let HF set this to an instance of transformers.DataCollatorWithPadding
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        compute_metrics=my_compute_metrics
    )

trainer.train()
model = trainer.model  # make sure to load_best_model_at_end=True!

Map:   0%|          | 0/439493 [00:00<?, ? examples/s]

Map:   0%|          | 0/109874 [00:00<?, ? examples/s]

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/439493 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/109874 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9824 [00:00<?, ? examples/s]

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text, premise, hypothesis. If text, premise, hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 439493
  Num Epochs = 20
  Instantaneous batch size per device = 80
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 1
  Total optimization steps = 109880
  Number of trainable parameters = 68097027
You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokeni

Step,Training Loss,Validation Loss,F1
128,1.120300,1.100493,0.165618
256,1.111700,1.115402,0.166644
384,1.105100,1.113874,0.166644
512,1.102600,1.099213,0.167734
640,1.100700,1.100036,0.167734
768,1.101300,1.098777,0.167734
896,1.100400,1.104862,0.166644
1024,1.099400,1.102981,0.167734
1152,1.100400,1.098987,0.165618
1280,1.101700,1.098798,0.165618


The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text, premise, hypothesis. If text, premise, hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 109874
  Batch size = 1280
Saving model checkpoint to /content/drive/My Drive/nlp244/assignments/quest-4/checkpoints/checkpoint-128
Configuration saved in /content/drive/My Drive/nlp244/assignments/quest-4/checkpoints/checkpoint-128/config.json
Model weights saved in /content/drive/My Drive/nlp244/assignments/quest-4/checkpoints/checkpoint-128/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp244/assignments/quest-4/checkpoints/checkpoint-128/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp244/assignments/quest-4/checkpoints/checkpoint-128/special_tokens_map.json
Deleting older c

In [ ]:
# run a final evaluation on the test set
trainer.evaluate(metric_key_prefix="test", eval_dataset=test_dataset)